In [2]:
!pip install face-recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566166 sha256=c7368e7b94e82918c77a85404f3b66b62fa3f47f250796fd3290d1c1664a7340
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models


In [5]:
!pip install dlib --no-cuda


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --no-cuda


In [7]:
!pip uninstall face-recognition -y  # Uninstall first
!pip install face-recognition --no-cuda

Found existing installation: face-recognition 1.3.0
Uninstalling face-recognition-1.3.0:
  Successfully uninstalled face-recognition-1.3.0

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --no-cuda


In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [15]:
!pip install face-recognition


In [16]:
# Patch face_recognition to remove CNN model (which requires GPU)
!sed -i '/cnn_face_detector =/,+1d' /usr/local/lib/python3.11/dist-packages/face_recognition/api.py


In [13]:
!pip install face-recognition --no-binary :all:
!pip install dlib
import cv2
import face_recognition
import numpy as np
import pandas as pd
import os
import datetime
import tkinter as tk
from tkinter import messagebox
from PIL import Image, ImageTk

# Encode faces from the dataset
def encode_faces_from_images():
    encoded_faces = {}
    for person_name in os.listdir('dataset'):
        person_path = os.path.join('dataset', person_name)
        if os.path.isdir(person_path):
            encodings = []
            for image_name in os.listdir(person_path):
                image_path = os.path.join(person_path, image_name)
                img = face_recognition.load_image_file(image_path)
                face_enc = face_recognition.face_encodings(img)
                if face_enc:
                    encodings.append(face_enc[0])
            if encodings:
                if person_name == "person1":
                    person_name = "Tejas.k"
                elif person_name == "person2":
                    person_name = "Sagar.s"
                encoded_faces[person_name] = encodings
    return encoded_faces

# Load known faces
known_encodings = encode_faces_from_images()
known_names = list(known_encodings.keys())

# Initialize attendance dictionaries
attendance = {}
attendance_images = {}

# Attendance marking function
def mark_attendance(name, frame, face_location):
    if name not in attendance:
        # Capture the time of attendance
        timestamp = datetime.datetime.now().strftime('%I:%M:%S %p')  # 12-hour format
        attendance[name] = timestamp

        # Save the image of the detected face
        top, right, bottom, left = face_location
        face_image = frame[top:bottom, left:right]
        face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)  # Convert to RGB for saving

        # Generate a unique filename for the image
        image_filename = f"photos/{name}_{datetime.datetime.now().strftime('%Y-%m-%d_%I-%M-%S %p')}.jpg"
        os.makedirs(os.path.dirname(image_filename), exist_ok=True)  # Ensure the directory exists
        cv2.imwrite(image_filename, cv2.cvtColor(face_image, cv2.COLOR_RGB2BGR))  # Save as BGR

        # Save the absolute path of the image in the attendance record
        attendance_images[name] = os.path.abspath(image_filename)

        # Inform the user
        messagebox.showinfo("Attendance Marked", f"Attendance marked for {name}")

# GUI setup
class AttendanceApp:
    def __init__(self, root):
        self.root = root
        self.root.title("AI-Based Attendance System")
        self.root.geometry("800x600")

        self.video_capture = None
        self.is_running = False

        # Create start and stop buttons
        self.start_button = tk.Button(root, text="Start Attendance", command=self.start_attendance)
        self.start_button.pack(pady=10)

        self.stop_button = tk.Button(root, text="Stop Attendance", command=self.stop_attendance)
        self.stop_button.pack(pady=10)

        # Video display label
        self.video_label = tk.Label(root)
        self.video_label.pack()

    # Start video capture and attendance
    def start_attendance(self):
        if not self.is_running:
            self.video_capture = cv2.VideoCapture(0)
            self.is_running = True
            self.update_frame()

    # Stop video capture and save attendance
    def stop_attendance(self):
        if self.is_running:
            self.is_running = False
            self.video_capture.release()
            # Save attendance to a CSV file
            self.save_attendance()

    # Save attendance to a CSV file
    def save_attendance(self):
        current_date = datetime.datetime.now().strftime('%Y-%m-%d')  # Current date
        attendance_data = [
            {
                "Name": name,
                "Date": current_date,  # Add the date
                "Time": time,
                "Photo": f'=HYPERLINK("{attendance_images.get(name, "")}", "View Photo")'
            }
            for name, time in attendance.items()
        ]
        attendance_df = pd.DataFrame(attendance_data)

        # Generate a unique filename based on date and time
        timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%I-%M-%S %p')
        filename = f"attendance_{timestamp}.csv"

        # Save to CSV
        attendance_df.to_csv(filename, index=False)
        messagebox.showinfo("Attendance Saved", f"Attendance saved to {filename}")

    # Update the video frame
    def update_frame(self):
        if self.is_running:
            ret, frame = self.video_capture.read()
            if not ret:
                self.stop_attendance()
                return

            # Process the frame
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(rgb_frame)
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            for face_encoding, face_location in zip(face_encodings, face_locations):
                name = "Unknown"
                for known_name, encodings in known_encodings.items():
                    matches = face_recognition.compare_faces(encodings, face_encoding, tolerance=0.6)
                    if any(matches):
                        name = known_name
                        mark_attendance(name, frame, face_location)
                        break

                # Draw a box around the face
                top, right, bottom, left = face_location
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # Convert frame for tkinter
            img = Image.fromarray(rgb_frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.imgtk = imgtk
            self.video_label.configure(image=imgtk)
            self.video_label.after(10, self.update_frame)

# Main function
if __name__ == "__main__":
    root = tk.Tk()
    app = AttendanceApp(root)
    root.mainloop()


RuntimeError: Error while calling cudaGetDevice(&the_device_id) in file /root/.cache/uv/sdists-v9/pypi/dlib/19.24.6/AJxE63M-IFcdC7-5Ufol7/src/dlib/cuda/gpu_data.cpp:204. code: 35, reason: CUDA driver version is insufficient for CUDA runtime version